In [7]:
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F

import utils
from models.generator import Generator
from models.discriminator  import Discriminator

import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import os

In [8]:
device = 'cuda' if torch.cuda.is_available() else  'cpu'
print(device)

cuda


In [14]:
generator = Generator()

generator.load_state_dict(torch.load('./checkpoints/trained_netG.pth'))

<All keys matched successfully>

In [15]:
generator

Generator(
  (down): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Sequential(
      (0): ReflectionPad2d((1, 1, 1, 1))
      (1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), bias=False)
    )
    (4): Sequential(
      (0): ReflectionPad2d((1, 1, 1, 1))
      (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), bias=False)
    )
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): Sequential(
      (0): ReflectionPad2d((1, 1, 1, 1))
      (1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), bias=False)
    )
    (8): Sequential(
      (0): ReflectionPad2d((1, 1, 1, 1))
      (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), bias=False)
    )
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_runni